In [ ]:
from icecream import ic
from rp import *

channel = JupyterDisplayChannel()
channel.display()

mask_channel = JupyterDisplayChannel()
mask_channel.display()


textured_model_channel = JupyterDisplayChannel()
textured_model_channel.display()

In [ ]:
import AlgoNotes as nw
import torch
from icecream import ic

if "device" not in vars():
    device = select_torch_device(prefer_used=True)

# uvl_image= rp.load_image('/root/CleanCode/Projects/deepfloyd_init_test/blender/BlenderOutputKevinSpinner/UV_Label_Exr/Image0155.exr',use_cache=True)
# uvl_image = rp.resize_image_to_fit(uvl_image,256,256,interp='nearest')
# texture_image = rp.load_image('https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png',use_cache=True)


if "texture_image" not in vars():
    tic()
    # texture_image=get_checkerboard_image(height=512*3,width=512*3)
    # texture_image = rp.as_float_image(rp.as_rgb_image(texture_image))
    # texture_image[:]=-1
    # texture_image=rp.as_torch_image(texture_image)
    # texture_image=torch.randn_like(texture_image)
    texture_image = torch.randn(4, 10000, 10000)
    noise_backdrop = torch.randn(4, 10000, 10000)  # Not too big
    texture_image = texture_image.to(device)
    noise_backdrop = noise_backdrop.to(device)
    # texture_image=texture_image.to(device)
    ptoc()

# texture_image= load_image('/Users/ryan/Downloads/KevinSpinner/frame_0002_point-cloud_geo_geo-retexture.jpg')


@memoized
def background_torch_randn(dims):
    return torch.randn(dims).to(device)


def warped_noise_from_uvl_path(path):
    uvl_image = rp.load_image(path, use_cache=True)
    # uvl_image = cv_resize_image(uvl_image, (512+1,)*2, 'nearest')
    uvl_image = cv_resize_image(uvl_image, (240 * 3, 135 * 3), "nearest")
    uvl_image = cv_resize_image(uvl_image, (114 * 6, 64 * 6), "nearest")
    uvl_image = as_torch_image(uvl_image).to(device)

    output = nw.uv_mapping_discretized(
        uv_image=uvl_image, tex_image=texture_image, w=10, device=device
    )
    ic(texture_image.shape, output.ryan_filter.shape)

    output.noisewarp = output.ryan_filter * output.area**0.5
    output.noisewarp_bad = output.ryan_filter * output.area**0.7

    out = output

    # rp.display_image(out.noisewarp/10+.5)
    ic(
        out.ryan_filter.min(),
        out.ryan_filter.max(),
        out.ryan_filter.mean(),
    )

    area = out.area
    mask = area == 0
    mask |= (
        uvl_image[2, 1:, 1:][None] != 0
    )  # Uses the L of UVL - main object has label 0
    ic(mask.sum())

    mask_channel.update(as_numpy_array(mask[0]))  # Display the mask...

    num_chan = out.noisewarp.shape[0]

    masked_noisewarp = out.noisewarp * ~(mask)
    masked_std = masked_noisewarp.std(correction=mask.sum() * num_chan)
    # masked_std = get_masked_std(out.noisewarp, mask)
    # ic(num_chan,mask.shape,masked_std)
    # print(masked_std)
    # masked_noisewarp = masked_noisewarp / 1.25 #TODO: Calculate this coeff correctly....the masked_std is wrong....
    masked_noisewarp = (
        masked_noisewarp / masked_std
    )  # TODO: Calculate this coeff correctly....the masked_std is wrong....
    background_noise = background_torch_randn(masked_noisewarp.shape)
    masked_background_noise = background_noise * mask
    output_noise = masked_noisewarp + masked_background_noise
    output_noise = as_numpy_image(output_noise)

    output_noise = (
        rp.cv_resize_image(output_noise, 1 / 6, "area") * 6
    )  # Variance-preserving rescale

    warped_noise_image = output_noise

    return output_noise

    return rp.as_numpy_image(out)


# ans = process_uvl_path('/root/CleanCode/Projects/deepfloyd_init_test/blender/BlenderOutputKevinSpinner/UV_Label_Exr/Image0155.exr')
# display_image(ans/10+.5)

# warped_noise_image / 10 + .5
from IPython.display import clear_output

output_folder = make_directory(
    get_unique_copy_path("noisewarp_outputs/KevinSpinnerAligned512")
)

ic.disable()
input_paths = sorted(
    rp_glob(
        "/root/CleanCode/Projects/deepfloyd_init_test/blender/BlenderOutputKevinSpinner/UV_Label_Exr/*.exr"
    )
)
input_paths = sorted(
    rp_glob(
        "/root/CleanCode/Projects/deepfloyd_init_test/blender/ProperSpinnerJul1/BlenderOutputKevinSpinner/UV_Label_Exr/*.exr"
    )
)[::-1]

for input_path in input_paths:
    warped_noise_image = warped_noise_from_uvl_path(input_path)
    clear_output()
    output_path = path_join(output_folder, get_file_name(input_path))
    print("IN:", input_path)
    print("OUT:", output_path)
    # display_image(warped_noise_image)

    saveable_image = warped_noise_image / 10 + 0.5
    channel.update(rp.as_rgb_image(saveable_image))
    # channel.update(rp.as_rgb_image(saveable_image))
    run_as_new_thread(
        save_image, saveable_image, output_path
    )  # Quantization doesn't matter too much...

At http://ryan-burgert-controlnet-tests.workbench.prod.netflix.net:54545
In /root/CleanCode/Projects/deepfloyd_init_test/noise_warping_python/noisewarp_outputs/KevinSpinner_copy16/*.exr
Use this code to view noise:

"""
import rp
import glob

def glob_search(query: str, replacements: dict) -> list:
    """
    Query is like "/path/to/{x:05}/image_*/{y}.png"
    Replacements is like {"x":5,"y":100}
    Returns a list of globbed paths
    """
    query = query.format(**replacements)
    paths = glob.glob(query)
    paths = sorted(paths)
    return paths

@rp.memoized
def load_image_bytes(path: str) -> bytes:
    image = rp.load_image(path, use_cache=True)
    # image = rp.resize_image_to_fit(image, height = 512,interp='area')
    image = rp.rotate_image(image, 90)
    title = '\n'.join(path.split('/')[-3:])
    image = rp.labeled_image(image, title, size=60)
    image_bytes = rp.encode_image_to_bytes(image, 'png', quality=95)
    return image_bytes

    # return rp.file_to_bytes(path) #Just loads the file raw
"""

In [ ]:
import AlgoNotes as nw
import torch
from icecream import ic

if "device" not in vars():
    device = select_torch_device(prefer_used=True)

if "kevin_texture_image" not in vars():
    kevin_texture_image = "/root/CleanCode/Projects/deepfloyd_init_test/blender/frame_0002_point-cloud_geo_geo-retexture.1001.jpg"
    kevin_texture_image = as_torch_image(
        as_float_image(as_rgb_image(load_image(kevin_texture_image)))
    ).to(device)


def textured_model_from_uvl_path(path):
    uvl_image = rp.load_image(path, use_cache=True)

    uvl_image = as_torch_image(uvl_image).to(device)

    output = nw.uv_mapping_discretized(
        uv_image=uvl_image, tex_image=kevin_texture_image, w=1, device=device
    )
    ic(texture_image.shape, output.ryan_filter.shape)

    out_image = output.linear

    is_blue = uvl_image[2, 1:, 1:][None] > 0
    out_image = torch.maximum(is_blue, out_image)

    out_image = as_numpy_image(out_image)
    return out_image


from IPython.display import clear_output

output_folder = make_directory(
    get_unique_copy_path("noisewarp_outputs/KevinSpinnerAlignedTextured")
)

ic.disable()
input_paths = sorted(
    rp_glob(
        "/root/CleanCode/Projects/deepfloyd_init_test/blender/BlenderOutputKevinSpinner/UV_Label_Exr/*.exr"
    )
)
input_paths = sorted(
    rp_glob(
        "/root/CleanCode/Projects/deepfloyd_init_test/blender/ProperSpinnerJul1/BlenderOutputKevinSpinner/UV_Label_Exr/*.exr"
    )
)[::-1]
for input_path in input_paths:
    textured_model_image = textured_model_from_uvl_path(input_path)
    clear_output()
    output_path = path_join(output_folder, get_file_name(input_path))
    print("IN:", input_path)
    print("OUT:", output_path)
    # display_image(warped_noise_image)

    saveable_image = textured_model_image
    textured_model_channel.update(saveable_image)
    run_as_new_thread(
        save_image, saveable_image, output_path
    )  # Quantization doesn't matter too much...

In [ ]:
kevin_texture_image = as_torch_image(
    as_float_image(as_rgb_image(load_image(kevin_texture_image)))
).to(device)

In [ ]:
del kevin_texture_image